# Se realizan importaciones

In [1]:
from Servicios.GoogleDriveService.GoogleDriveService import GoogleDriveService
from Servicios.AdvanaService.AdvanaService import AdvanaService
from Servicios.OrquestadorService.OrchestrateService import OrchestrateService

from Utilidades.GlobalUtility import ObtenerDataPorHTTPRequest
import Utilidades.GlobalUtility as Utility

from pathlib import Path

from q1_memory import q1_memory
from q1_time import q1_time

from q2_memory import q2_memory
from q2_time import q2_time

from q3_memory import q3_memory
from q3_time import q3_time

import json

print("Cargando spark.. puede tardar un poco")

Cargando spark.. puede tardar un poco


# Se definen variables de Buckets de GCP

In [2]:
IdContenedorBronze = "1yQCQ3BhNPdXiz890KTF9iNB5qdeTATlV" #Bronce
IdContenedorGold = "1TMq41pVhsEDK1IUP4kQHzf7aIsagVihE" #Gold

URLRequestRawDataZip = "https://drive.usercontent.google.com/download?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&export=download&authuser=0&confirm=t&uuid=d3ab3e9e-0e9e-4dfe-945a-a73286456315&at=APZUnTUiCQBVxpETIVQcpNAPpA7U%3A1720637781554"
NombreArchivo = "data.json"

# Se definen variables de Advana

In [3]:
JSONObjEntregaAdvana = {"name": "TDD (Pruebas) EnviarPeticionHTTP() Advana Fmateu",
           "mail": "francisco.mateu.araneda@gmail.com Aun en desarrollo",
           "github_url": "https://github.com/xFrancisc0/PrototipoDE"}
           
URLRequestEntregaAdvana = "https://advana-challenge-check-api-cr-k4hdbggvoq-uc.a.run.app/data-engineer"

# Se instancian los servicios de la aplicación y se inyectan en el orquestador
El objetivo es una vez que se instancie el orquestador, que se orqueste una ingesta de data desde https://drive.google.com/file/d/1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis/view?usp=sharing y que se guarde en la zona bronze del "Datalake" de prototipo Es una URL pública: https://drive.google.com/drive/u/2/folders/1nVbnD0pgYnAeym3lUoNna5cE7goN2U5N (Fue desplegado este Datalake para ser visto desde el exterior sin permisos de edición. Esto es para mostrar el funcionamiento del presente proyecto)

Posteriormente, el orquestador cargara la data desde la zona bronce, se le aplicarán todos los procedimientos y se guardará en la zona Gold.

In [4]:
# Crear instancias de servicios e inyectarlos en el orquestador para mayor simplicidad
google_drive_service = GoogleDriveService()
advana_service = AdvanaService()

# Crear instancia de OrchestrateService, inyectando GoogleDriveService
orchestrate_service = OrchestrateService(google_drive_service, advana_service)

#Destruir objetos que no se ocuparan
del google_drive_service
del advana_service

# Se obtiene la raw data y se almacena en zona bronze
Para ver el datalake: https://drive.google.com/drive/u/2/folders/1nVbnD0pgYnAeym3lUoNna5cE7goN2U5N

In [5]:

RawDataInput = Utility.ObtenerDataPorHTTPRequest(URLRequestRawDataZip, 'GET', 'JSON', True)

#Cargar raw data a zona bronze
orchestrate_service.google_drive_service.subir_archivo_a_contenedor(IdContenedorBronze, NombreArchivo, RawDataInput)

#Destruir objeto RawDataInput
del RawDataInput


Obteniendo JSON POR HTTP Request
Archivo JSON descomprimido y cargado exitosamente.
Subiendo data a Google Drive
Archivo data.json ya existente en google cloud con ID 11UpXKbNzPN8AvyUJVt1ED6ZdHFeyzARW eliminado (overwrite).
Archivo 'data.json' subido correctamente. ID: 1zRFZTw8OhuHmDsC04YP9SnggX6BEvaaH


# Se obtiene la data de la zona de bronze y se guarda en TemporalBlobs. Se elimina de la memoria.

In [6]:
#Cargar en memoria de zona bronze
BronzeMetadata = orchestrate_service.google_drive_service.listar_archivos_en_contenedorSTR(IdContenedorBronze)
BronzeDataInput = orchestrate_service.google_drive_service.descargar_archivo_porid(BronzeMetadata[0]["id"])

# Crear una instancia de Path para la ruta del archivo
archivo_path = Path(f'TemporalBlobs/{NombreArchivo}')
archivo_path_parcial = f'TemporalBlobs/{NombreArchivo}'

# Asegurarse de que el directorio existe
archivo_path.parent.mkdir(parents=True, exist_ok=True)

# Escribir los datos al archivo
with open(archivo_path, 'wb') as f:
    f.write(BronzeDataInput)  # Escribir los datos en formato de bytes

print(f"Archivo bronze guardado en: {archivo_path}")

Archivos encontrados:  [{'kind': 'drive#file', 'mimeType': 'application/json', 'id': '1zRFZTw8OhuHmDsC04YP9SnggX6BEvaaH', 'name': 'data.json'}]
Descargando 25%.
Descargando 51%.
Descargando 77%.
Descargando 100%.
Archivo bronze guardado en: TemporalBlobs\data.json


# Se procede a realizar las transformaciones requeridas

In [7]:
print("")
print("Q1: Las top 10 fechas donde hay más tweets y sus usuarios: ")
print("===========================================================")
q1_m = q1_memory(archivo_path_parcial)
#q1_t = q1_time(archivo_path_parcial)
print(q1_m)


print("")
print("Los top 10 emojis más usados con su respectivo conteo: ")
print("===========================================================")
q2_m = q2_memory(archivo_path_parcial)
#q2_t = q2_time(archivo_path_parcial)
print(q2_m)


print("")
print("Q3: El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos:")
print("===========================================================")
q3_m = q3_memory(archivo_path_parcial)
#q3_t = q3_time(archivo_path_parcial)
print(q3_m)


Q1: Las top 10 fechas donde hay más tweets y sus usuarios: 
Q1: Iniciando


PySparkTypeError: [NOT_STR_OR_LIST_OF_RDD] Argument `path` should be a str or list[RDD], got WindowsPath.

# El dataset posterior a sus transformaciones se guarda en la zona gold del Datalake
Para ver el datalake: https://drive.google.com/drive/u/2/folders/1nVbnD0pgYnAeym3lUoNna5cE7goN2U5N

In [ ]:
orchestrate_service.google_drive_service.subir_archivo_a_contenedor(IdContenedorGold, "q1.json", q1_m)
orchestrate_service.google_drive_service.subir_archivo_a_contenedor(IdContenedorGold, "q2.json", q2_m)
orchestrate_service.google_drive_service.subir_archivo_a_contenedor(IdContenedorGold, "q3.json", q3_m)

#Destruir objetos
del q1_m
#del q1_t
del q2_m
#del q2_t
del q3_m
#del q3_t

# Informar entrega por advana

In [ ]:
print("Informa de entrega a advana")
JSONObj = {"name": "TDD (Pruebas) EnviarPeticionHTTP() Advana Fmateu",
           "mail": "francisco.mateu.araneda@gmail.com",
           "github_url": "https://github.com/xFrancisc0/PrototipoDE"
          }
URLRequest = "https://advana-challenge-check-api-cr-k4hdbggvoq-uc.a.run.app/data-engineer"
self.advana_service.EnviarPeticionHTTP(JSONObj, URLRequest, "POST", "REST")
orchestrate_service.advana_service